In [3]:
# NEW CODE - Script to create a master NFL DB schema view in Excel (with column position)

import sqlite3
import pandas as pd

# === CONFIG ===
DB_PATH = r"C:\Users\tx400\SQL_Downloads\nfl_data_db.sqlite"  # change if needed
OUTPUT_FILE = "nfl_schema_overview.xlsx"

# === CONNECT TO DATABASE ===
conn = sqlite3.connect(DB_PATH)

# === GET LIST OF TABLES ===
tables = pd.read_sql(
    "SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;", conn
)["name"]

# === COLLECT SCHEMA INFORMATION ===
schema = []
for t in tables:
    cols = pd.read_sql(f"PRAGMA table_info({t});", conn)
    for _, row in cols.iterrows():
        schema.append({
            "table_name": t,
            "column_name": row["name"],
            "data_type": row["type"],
            "column_position": row["cid"] + 1  # NEW CODE - 1-based column position
        })

df_schema = pd.DataFrame(schema)

# === LONG FORM (Searchable) ===
df_long = df_schema.sort_values(["table_name", "column_position"])

# === PIVOTED VIEW (Alignment Sheet) ===
pivot = (
    df_schema
    .assign(present="✅")  # ✅ marker for presence
    .pivot_table(index="column_name", columns="table_name", values="present", aggfunc="first", fill_value="")
    .reset_index()
)

# === SAVE TO EXCEL ===
with pd.ExcelWriter(OUTPUT_FILE, engine="openpyxl") as writer:
    df_long.to_excel(writer, sheet_name="Long_Form_Schema", index=False)
    pivot.to_excel(writer, sheet_name="Column_Alignment", index=False)

conn.close()

print(f"✅ Schema overview saved to: {OUTPUT_FILE}")


✅ Schema overview saved to: nfl_schema_overview.xlsx
